### Abstract

This notebook is intended to showcase how to use the MNL (Multinomial Logistic Regression) model to predict the booking probability for each option within a session.

One can find the sample training and testing data under the `data` folder.

In [1]:
# import the model and all the auxiliary functions
from MNL import *
from MNL_plus import *
from Mint import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import pprint 
pp = pprint.PrettyPrinter(indent=4)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,6
rcParams['figure.dpi'] = 100
rcParams['savefig.dpi'] = 100

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-r5ol5wj4 because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
# To save output into txt file
from contextlib import redirect_stdout

In [ ]:
TRAIN_CONFIG = {
    #'MNL_features': MNL_features,
    
    # when absent, by default, use all the features within the training data
    #'alter_features': MNL_features,
    #'session_features'
    
    # options: BinaryCrossEntropy, MaxLogLikelihood
    #'loss':  'MaxLogLikelihood',
    'loss':  'MaxLogLikelihood',
    
    'expand': False,
    
    'optimizer': 'Adam',  # options:  Adam, RMSprop, SGD, LBFGS.
    # Adam would converge much faster
    # LBFGS is a very memory intensive optimizer (it requires additional param_bytes * (history_size + 1) bytes).
    # If it doesn’t fit in memory try reducing the history size, or use a different algorithm.
    # By default, history_size == 100
    'learning_rate': 0.1, # Applicable to Adam, SGD, and LBFGS
    # The learning_rate parameter seems essential to LBFGS, which converges in two epochs.
    #  So far, learning_rate == 0.1 seems to be ok for LBFGS
    
    #'momentum': 0.9,  # applicable to SGD, RMSprop
    'momentum': 0.01,  # applicable to SGD, RMSprop
    
    # The resulting model seems to be more balanced, i.e. no extreme large/small weights,
    #  although one might not have the most ideal performance, i.e. high top_5_rank etc.
    'weight_decay': 0, # Applicable to Adam, RMSprop and SGD
    

    # indicates the number of sessions included in each batch
    'batch_size': 500,
    
    #maximum number of epochs
    'epochs': 20,
    
    #tolerance for early stopping
    'early_stop_min_delta': 1e-4,
    'patience': 5,
    
    #if able to use GPU (unfortunately I am not able to do this)
    'gpu': False,  # luckily, running on GPU is faster than CPU in this case.
    
    # level of logging, 0: no log,  1: print epoch related logs;  2: print session related logs
    'verbose': 1,
    
    # Adding the regularization degredates the performance of model
    #   which might suggests that the model is still underfitting, not overfitting.
    'l1_loss_weight': 0,  # e.g. 0.001 the regularization that would marginalize the weights
    'l2_loss_weight': 0,
    
    # flag indicates whether to save gradients during the training
    'save_gradients': False

}


In [ ]:
# set random seed for reproduceability
np.random.seed(17)
torch.manual_seed(17)

df_train = pd.read_csv('data/train_SINBKK_RT_B.csv')

# Create a brand-new model
from contextlib import redirect_stdout

model_tuple, loss_list = run_training(df_training=df_train, train_config=TRAIN_CONFIG)

# Continue training on the existing model
model_tuple, loss_list = run_training(df_training=df_train, train_config=TRAIN_CONFIG, model_tuple=model_tuple)


# unzip the tuple
(model, loss, optimizer) = model_tuple


# plot the evolution of loss
plot_loss(loss_list)

### Model Stats

In this section, we calculate some statistics in the train data

In [ ]:
# Test the model with the testing data
# Calculate the probability and the trank of the chosen alternative
train_stats = validate(model, df_train, TRAIN_CONFIG)
train_stats.head()

Summarize the testing results into a list of KPIs, such as:

- *mean_probability*: the average probability of the predicted alternative among all sessions


- *top_5_rank_quantile*: the percentile of sessions where the probability of the predicted alternative is among the top 5.


- *AIC*: Akaike Information Criterion, which offers an estimate of the relative information lost when a given model is used to represent the process that generated the data.

In [ ]:
summarize_KPIs(train_stats, len(TRAIN_CONFIG['MNL_features']))

## Implementation with expand option 

In this section, I show how to use the expand option. You will see that there are three datasets this time (take a look at the dummy do file in the data folder to see how they are constructed)

In [ ]:
session_id=['session_id']
alter_id=['alter_id']
alter_features=
session_features=
choice_groups=
session_alter_features=

In [4]:
#define choice-alternative function
def session_alter(data):
    return data['reco_contains_mh']*data['deptime_outbound_sin4p']+data['deptime_outbound_cos4p']

In [5]:
from nmlogit_config	 import *
config['session_id']='session_id'
config['alter_id']='alter_id'
config['alter_features']=['reco_contains_mh', 'reco_contains_tg', 'reco_contains_pg', 'reco_contains_sq', 'reco_contains_vn', 'reco_contains_cx', 'reco_contains_od']
config['session_features']=['deptime_outbound_sin2p', 'deptime_outbound_sin4p', 'deptime_outbound_cos2p', 'deptime_outbound_cos4p', 'deptime_inbound_sin2p', 'deptime_inbound_sin4p', 'deptime_inbound_cos2p', 'deptime_inbound_cos4p']
config['choice_groups']=['group','group2']
config['session_alter_features']=[]
config['extra_choice_features']=[]
config['session_alter_fn']=[]
config['drop_vars']=[]
config['batch_size']=7788
config['loss']='BinaryCrossEntropy'

In [12]:
choices = pd.read_csv('data/train_SINBKK_RT_B_choices.csv')[['session_id','alter_id','choice','group','group2']]
alter_data = pd.read_csv('data/train_SINBKK_RT_B_alter_id.csv')
session_data = pd.read_csv('data/train_SINBKK_RT_B_session_id.csv')

In [13]:
model_tuple, loss_list = run_training(df_training=choices, train_config=config, alter_data=alter_data, session_data=session_data)
(model, loss, optimizer) = model_tuple

Num features: 15
All features: ['reco_contains_mh', 'reco_contains_tg', 'reco_contains_pg', 'reco_contains_sq', 'reco_contains_vn', 'reco_contains_cx', 'reco_contains_od', 'deptime_outbound_sin2p', 'deptime_outbound_sin4p', 'deptime_outbound_cos2p', 'deptime_outbound_cos4p', 'deptime_inbound_sin2p', 'deptime_inbound_sin4p', 'deptime_inbound_cos2p', 'deptime_inbound_cos4p']
Alternative Features: ['reco_contains_mh', 'reco_contains_tg', 'reco_contains_pg', 'reco_contains_sq', 'reco_contains_vn', 'reco_contains_cx', 'reco_contains_od']
Session Features: ['deptime_outbound_sin2p', 'deptime_outbound_sin4p', 'deptime_outbound_cos2p', 'deptime_outbound_cos4p', 'deptime_inbound_sin2p', 'deptime_inbound_sin4p', 'deptime_inbound_cos2p', 'deptime_inbound_cos4p']
Loaded but dropped features: []
Group IDs: ['group', 'group2']
Batch_id: 0/1: Size=7788 expanded to 486750
epoch: 0  loss: 0.07682089745409494 best_loss: 1000000000000000.0
Batch_id: 0/1: Size=7788 expanded to 486750
epoch: 1  loss: 0.076

In [75]:
model.get_params().grad_tensor()

AttributeError: 'Parameter' object has no attribute 'grad_tensor'

In [60]:
gradient2=model.get_params().grad

In [56]:
with torch.no_grad():
        gradient = [item.grad for item in model.parameters()]
            

In [71]:
model.grad_tensor

AttributeError: 'MNL' object has no attribute 'grad_tensor'

In [58]:
gradient[0]

tensor([[-1.2265e-06,  1.0112e-05,  0.0000e+00, -6.1684e-06,  0.0000e+00,
         -8.9087e-07,  0.0000e+00, -6.8508e-18,  1.2993e-18, -3.6253e-19,
          8.1315e-19, -2.1176e-18,  1.1922e-18,  4.2352e-20,  6.8483e-19]])

Get statistics

In [107]:
np.random.seed(20)
x=np.random.rand(5000)
y=np.random.rand(5000)
z=np.random.rand(5000)

In [114]:
len(np.where((y>x) & (x<y),1,0))

5000

In [90]:
x[3]=5

In [91]:
x

[1, 2, 0, 5, 0]

In [88]:
np.where(choices['choice']==0,1,2)

array([1, 1, 1, ..., 1, 1, 1])

In [8]:
choices = pd.read_csv('data/train_SINBKK_RT_B_choices.csv')[['session_id','alter_id','choice','group','group2']]

In [7]:
test_results = test_model(model=model, df_testing=choices, train_config=config, alter_data=alter_data, session_data=session_data)
test_results.head()

NameError: name 'model' is not defined

In [ ]:
choices = pd.read_csv('data/train_SINBKK_RT_B_choices.csv')[['session_id','alter_id','choice','group','group2']]
# Test the model with the testing data
# Calculate the probability and the trank of the chosen alternative
train_stats = validate(model=model, df_testing=choices, train_config=config, alter_data=alter_data, session_data=session_data)
train_stats.head()

Summarize the testing results into a list of KPIs, such as:

- *mean_probability*: the average probability of the predicted alternative among all sessions


- *top_5_rank_quantile*: the percentile of sessions where the probability of the predicted alternative is among the top 5.


- *AIC*: Akaike Information Criterion, which offers an estimate of the relative information lost when a given model is used to represent the process that generated the data.

In [ ]:
summarize_KPIs(train_stats, len(config['MNL_features']))

In [ ]:
import torch

In [ ]:
gradient[0]

In [ ]:
torch.ger(gradient[0],gradient[0])

In [ ]:
from scipy.stats import norm
from operator import truediv

b=model.get_params()
gradient=model.get_params().grad
hessian=torch.mm(torch.t(gradient),gradient)
se = torch.sqrt(torch.diagonal(hessian,0))
t=torch.div(b,se)


In [ ]:
import torch  
x=torch.tensor(2.0, requires_grad=True)  
z=torch.tensor(4.0, requires_grad=True)  
y=x**2+z**3  
y.backward()  
x.grad  
z.grad  

In [ ]:
x.grad

In [ ]:
v1 = torch.arange(1., 5.)
v2 = torch.arange(1., 4.)
torch.outer(v1, v2)

In [ ]:
torch.outer(gradient,gradient)

In [ ]:
b=b.tolist()
se=se.tolist()
gradient=gradient.tolist()
hessian=hessian.tolist()
t=t.tolist()

In [ ]:
p=1-norm.cdf(t)

In [ ]:
torch.diagonal(hessian).size()

In [ ]:
model

In [ ]:
model.get_params().grad[0]

In [ ]:
a = [[1], [2],[3],[4]]

In [ ]:
a

In [ ]:
import numpy as np

In [79]:
x=[1,2,0,4,0]

In [81]:
np.where(x==0,1,0)

array(0)

In [ ]:
with torch.no_grad():
            gradient = torch.stack([(item.grad ** 2).sum() for item in model.parameters()]).sum()

In [78]:
import numpy as np
from torch.autograd import grad

def sigmoid(x):
    return 0.5 * (np.tanh(x / 2.) + 1)

def logistic_predictions(weights, inputs):
    # Outputs probability of a label being true according to logistic model.
    return sigmoid(np.dot(inputs, weights))

def training_loss(weights):
    # Training loss is the negative log-likelihood of the training labels.
    preds = logistic_predictions(weights, inputs)
    label_probabilities = preds * targets + (1 - preds) * (1 - targets)
    return -np.sum(np.log(label_probabilities))

# Build a toy dataset.
inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])

# Define a function that returns gradients of training loss using Autograd.
training_gradient_fun = grad(training_loss)

# Optimize weights using gradient descent.
weights = np.array([0.0, 0.0, 0.0])
print("Initial loss:", training_loss(weights))
for i in range(100):
    weights -= training_gradient_fun(weights) * 0.01

print("Trained loss:", training_loss(weights))

TypeError: grad() missing 1 required positional argument: 'inputs'